In [1]:
import jqdatasdk as jq
import pandas as pd

In [2]:
jq.auth('13918852005','jxjslsc,.?2')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [17]:
# 股指换手率、交易量数据 #
df = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322001,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))

In [18]:
df.index = df['date']

In [19]:
df.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_motion090101-190227.csv',index= True)

In [22]:
df1 = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322005,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))
df1.index = df1['date']
df1.drop(['turnover_ratio'],axis=1,inplace=True)
df1['turnover_ratio'] = df1.money/df1.circulating_market_cap*100
df1.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_motion090101-190227.csv',index= True)

In [25]:
df2 = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322007,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))
df2.index = df2['date']
df2.drop(['turnover_ratio'],axis=1,inplace=True)
df2['turnover_ratio'] = df2.money/df2.circulating_market_cap*100
df2.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/CYBZ_motion090101-190227.csv',index= True)

In [30]:
# 交易日前一日新闻联播内容股市相关提到次数 全跑一遍 #
# 关键词词库
df_temp0 = jq.finance.run_query(jq.query(jq.finance.CCTV_NEWS).filter(
        jq.finance.CCTV_NEWS.day>='2009-01-01',
        jq.finance.CCTV_NEWS.title.like('%%金融%%')
    ))

for key_word in ['股市','证监会','证券','股票']:
    df_temp1 = jq.finance.run_query(jq.query(jq.finance.CCTV_NEWS).filter(
        jq.finance.CCTV_NEWS.day>='2009-01-01',
        jq.finance.CCTV_NEWS.title.like('%%'+key_word+'%%'),
    ))
    df_temp0 = pd.concat([df_temp0,df_temp1],axis=0)
    
for key_word in ['金融','股市','证监会','证券','股票']:
    df_temp2 = jq.finance.run_query(jq.query(jq.finance.CCTV_NEWS).filter(
        jq.finance.CCTV_NEWS.day>='2009-01-01',
        jq.finance.CCTV_NEWS.content.like('%%'+key_word+'%%'),
    ))
    df_temp0 = pd.concat([df_temp0,df_temp2],axis=0)

In [31]:
df_new_cnt = df_temp0.groupby(df_temp0['day']).count()[['content']]

In [32]:
# 今天的新闻联播对明天有影响，因此日期+1
from datetime import datetime, date, timedelta
day_list_pre1 = []
for i in list(df_new_cnt.index):
    day_temp = i + timedelta(1)
    day_list_pre1.append(str(day_temp))

In [33]:
df_new_cnt.index = day_list_pre1

In [34]:
df_new_cnt_all = pd.DataFrame(columns=['temp'],index=pd.date_range('1/1/2009','2/1/2019'))
df_new_cnt_all = pd.merge(df_new_cnt_all, df_new_cnt, how = 'outer' ,left_index=True, right_index=True)
df_new_cnt_all = df_new_cnt_all[['content']]
df_new_cnt_all = df_new_cnt_all.fillna(0)

In [35]:
# datetime.strptime(df_new_cnt_all.index[i], '%Y-%m-%d').weekday() == 5 or datetime.strptime(df_new_cnt_all.index[i], '%Y-%m-%d').weekday() == 6

In [36]:
for i in range(len(df_new_cnt_all)-1):
    if df_new_cnt_all.index[i].weekday() == 5 or df_new_cnt_all.index[i].weekday() == 6:
        df_new_cnt_all.iloc[i+1,0] += df_new_cnt_all.iloc[i,0]

In [37]:
df_new_cnt_all['content'] = df_new_cnt_all.rolling(7).sum() # 过去一周新闻总和 - 持续影响，减少0

In [38]:
df_new_cnt_all.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news_cnt_090101-190201.csv',index= True)  
# 最后用inner merge将df_new_cnt_all合并到总表中，会自动删除=5and=6(周六、周日新闻，留下周一（实际是周五+周六+周日）)

In [23]:
df_new.iloc[3093,0]

'-1'

In [25]:
# 爬虫形式自行获取的过去两周新闻搜索量
df_new = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news091201-190201.csv',index_col=0)

In [26]:
import re
new_cnt_lst=[]
for i in range(len(df_new)):
    try:
        new_cnt = re.findall('找到相关新闻约(.*?)篇',df_new.iloc[i,0])[0]
    except:
        new_cnt = re.findall('找到相关新闻(.*?)篇',df_new.iloc[i,0])[0]
    new_cnt_lst.append(int(new_cnt.replace(',','')))

In [27]:
import numpy as np
df_new['content'] = new_cnt_lst
df_new['content'] = df_new['content'].apply(lambda x: np.log(x+1))

In [28]:
df_new_save = df_new[['content']]
df_new_save.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news_cnt_090101-190201.csv',index= True) # 会覆盖JQData的数据！